# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import re
import pickle
from sqlalchemy import create_engine
from nltk import word_tokenize, pos_tag, ne_chunk, WordNetLemmatizer
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
# import category_encoders as ce

In [2]:
# load data from database
engine = create_engine('sqlite:///../data/Disaster_Response.db')
# df = pd.read_sql("SELECT * FROM Disaster_Response", engine) # alternative
df = pd.read_sql_table("Disaster_Response", engine)
X = df.loc[:,'message':'genre']
Y = df.loc[:,'related':]
X.shape, Y.shape

((26144, 3), (26144, 36))

### 2. Write a tokenization function to process your text data

In [3]:
# regex pattern for detecting a url
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

# define tokenizer
def tokenize(text):
    # get list of all urls using regex
    detected_urls = re.findall(url_regex,text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url,'urlplaceholder')

    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [4]:
# test the tokenizer
text=X.message[0]
print(tokenize(text))

['weather', 'update', '-', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
# simple 2-step pipeline for text-processing and classification
pipeline = Pipeline([
    ('text',TfidfVectorizer(stop_words = 'english', tokenizer = tokenize)),
    ('clf',RandomForestClassifier(random_state=1234,n_jobs=-1))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
%%time

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=1234)

# train the pipeline
pipeline.fit(X_train.message, y_train)

# predict on test data
y_pred = pipeline.predict(X_test.message)

//anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


CPU times: user 24.9 s, sys: 894 ms, total: 25.8 s
Wall time: 11.8 s


In [7]:
y_test.shape, y_pred.shape

((6536, 36), (6536, 36))

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
# show prediction results
def show_result(pred, test = y_test):
    for i, col in enumerate(Y.columns):
        print('\033[1m'+col+'\033[0m', '\n', metrics.classification_report(test.iloc[:,i], pred[:,i]))

In [9]:
show_result(y_pred)

related 
               precision    recall  f1-score   support

           0       0.61      0.55      0.58      1563
           1       0.86      0.89      0.87      4920
           2       0.50      0.06      0.10        53

    accuracy                           0.80      6536
   macro avg       0.66      0.50      0.52      6536
weighted avg       0.80      0.80      0.80      6536

request 
               precision    recall  f1-score   support

           0       0.89      0.98      0.93      5428
           1       0.79      0.41      0.54      1108

    accuracy                           0.88      6536
   macro avg       0.84      0.69      0.74      6536
weighted avg       0.87      0.88      0.87      6536

offer 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6516
           1       0.00      0.00      0.00        20

    accuracy                           1.00      6536
   macro avg       0.50      0.50      0.50   

//anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


shops 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6511
           1       0.00      0.00      0.00        25

    accuracy                           1.00      6536
   macro avg       0.50      0.50      0.50      6536
weighted avg       0.99      1.00      0.99      6536

aid_centers 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6471
           1       0.00      0.00      0.00        65

    accuracy                           0.99      6536
   macro avg       0.50      0.50      0.50      6536
weighted avg       0.98      0.99      0.99      6536

other_infrastructure 
               precision    recall  f1-score   support

           0       0.95      1.00      0.98      6239
           1       0.00      0.00      0.00       297

    accuracy                           0.95      6536
   macro avg       0.48      0.50      0.49      6536
weighted avg       0.91      

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
# for gird-search MultiOutputClassifier is used 
# for extending classifiers that do not natively support multi-target classification 

pipeline = Pipeline([
    ('text',TfidfVectorizer(stop_words = 'english', tokenizer = tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=1234, n_jobs=-1)))
])

In [11]:
# peek at tunable parameters
pipeline.get_params()

{'memory': None,
 'steps': [('text',
   TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.float64'>, encoding='utf-8',
                   input='content', lowercase=True, max_df=1.0, max_features=None,
                   min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                   smooth_idf=True, stop_words='english', strip_accents=None,
                   sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x1a2670b488>, use_idf=True,
                   vocabulary=None)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=None,
                                                          max_features='auto',


In [12]:
# setup tunable parameters
parameters = {
              'text__ngram_range': [(1,2),(1,3)],
              'text__use_idf': (True, False),
              'clf__estimator__n_estimators': [20,40],
#               'clf__estimator__min_samples_split': [2,4,8]
              }

In [13]:
%%time
# setup gridsearch
cv = GridSearchCV(pipeline, param_grid=parameters,return_train_score=True, verbose=5) 
# fit the data
cv.fit(X_train.message, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] clf__estimator__n_estimators=20, text__ngram_range=(1, 2), text__use_idf=True 


//anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
//anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


[CV]  clf__estimator__n_estimators=20, text__ngram_range=(1, 2), text__use_idf=True, score=(train=0.941, test=0.238), total=  40.2s
[CV] clf__estimator__n_estimators=20, text__ngram_range=(1, 2), text__use_idf=True 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   50.4s remaining:    0.0s
//anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


[CV]  clf__estimator__n_estimators=20, text__ngram_range=(1, 2), text__use_idf=True, score=(train=0.943, test=0.241), total=  39.8s
[CV] clf__estimator__n_estimators=20, text__ngram_range=(1, 2), text__use_idf=True 


...

//anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


[CV]  clf__estimator__n_estimators=40, text__ngram_range=(1, 3), text__use_idf=False, score=(train=0.981, test=0.247), total= 1.6min


[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 30.6min finished
//anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


CPU times: user 30min 46s, sys: 1min, total: 31min 46s
Wall time: 32min 30s


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('text',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                     

In [14]:
cv.best_score_

0.2483170134638923

In [15]:
cv.best_params_

{'clf__estimator__n_estimators': 40,
 'text__ngram_range': (1, 2),
 'text__use_idf': True}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
# predict and show the result
y_pred_tuned = cv.predict(X_test.message)

show_result(y_pred_tuned)

related 
               precision    recall  f1-score   support

           0       0.70      0.36      0.47      1563
           1       0.82      0.95      0.88      4920
           2       0.33      0.04      0.07        53

    accuracy                           0.80      6536
   macro avg       0.62      0.45      0.47      6536
weighted avg       0.79      0.80      0.78      6536

request 
               precision    recall  f1-score   support

           0       0.90      0.98      0.94      5428
           1       0.81      0.48      0.60      1108

    accuracy                           0.89      6536
   macro avg       0.86      0.73      0.77      6536
weighted avg       0.89      0.89      0.88      6536

offer 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6516
           1       0.00      0.00      0.00        20

    accuracy                           1.00      6536
   macro avg       0.50      0.50      0.50   

//anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


aid_centers 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6471
           1       0.00      0.00      0.00        65

    accuracy                           0.99      6536
   macro avg       0.50      0.50      0.50      6536
weighted avg       0.98      0.99      0.99      6536

other_infrastructure 
               precision    recall  f1-score   support

           0       0.95      1.00      0.98      6239
           1       0.33      0.00      0.01       297

    accuracy                           0.95      6536
   macro avg       0.64      0.50      0.49      6536
weighted avg       0.93      0.95      0.93      6536

weather_related 
               precision    recall  f1-score   support

           0       0.86      0.96      0.91      4722
           1       0.85      0.60      0.70      1814

    accuracy                           0.86      6536
   macro avg       0.86      0.78      0.81      6536
weighted avg       

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [17]:
# import sklearn
# sklearn.__version__

In [18]:
%%time

# add categorical feature through the pipeline
# the performance may increase slightly but this approuch currently does not work with the frontend web app
# and the generated model could takeup substantial storage space
preprocessor = ColumnTransformer(transformers=[
     ('text', TfidfVectorizer(tokenizer = tokenize, 
                              stop_words = 'english', 
                              ngram_range=cv.best_params_['text__ngram_range'],
                              use_idf=cv.best_params_['text__use_idf']),
      'message'), # 'message' VS ['message'] why error??
     ('cate', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['genre'])
])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
model = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier(random_state=0,
                                                            n_estimators=cv.best_params_['clf__estimator__n_estimators'],
                                                            n_jobs=-1
                                                   ))])

# fit X_train isntead of X_train.message
model.fit(X_train, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


CPU times: user 3min 42s, sys: 1.64 s, total: 3min 43s
Wall time: 35.4 s


Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('text',
                                                  TfidfVectorizer(analyzer='word',
                                                                  binary=False,
                                                                  decode_error='strict',
                                                                  dtype=<class 'numpy.float64'>,
                                                                  encoding='utf-8',
                                                                  input='content',
                                                                  lowercase=True,
                                                                  max_df=1.0,
                           

In [19]:
# predict and show the result
y_pred_multi_feat = model.predict(X_test)

show_result(y_pred_multi_feat)

related 
               precision    recall  f1-score   support

           0       0.68      0.48      0.56      1563
           1       0.84      0.93      0.89      4920
           2       0.67      0.08      0.14        53

    accuracy                           0.82      6536
   macro avg       0.73      0.50      0.53      6536
weighted avg       0.80      0.82      0.80      6536

request 
               precision    recall  f1-score   support

           0       0.90      0.98      0.94      5428
           1       0.83      0.48      0.61      1108

    accuracy                           0.89      6536
   macro avg       0.87      0.73      0.77      6536
weighted avg       0.89      0.89      0.88      6536

offer 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6516
           1       0.00      0.00      0.00        20

    accuracy                           1.00      6536
   macro avg       0.50      0.50      0.50   

//anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


shops 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6511
           1       0.00      0.00      0.00        25

    accuracy                           1.00      6536
   macro avg       0.50      0.50      0.50      6536
weighted avg       0.99      1.00      0.99      6536

aid_centers 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6471
           1       0.00      0.00      0.00        65

    accuracy                           0.99      6536
   macro avg       0.50      0.50      0.50      6536
weighted avg       0.98      0.99      0.99      6536

other_infrastructure 
               precision    recall  f1-score   support

           0       0.95      1.00      0.98      6239
           1       0.00      0.00      0.00       297

    accuracy                           0.95      6536
   macro avg       0.48      0.50      0.49      6536
weighted avg       0.91      

### 9. Export your model as a pickle file

###  Create a manual model for demo

In [35]:
%%time

model_m = Pipeline([
    ('text',TfidfVectorizer(stop_words = 'english', tokenizer = tokenize, use_idf=True, ngram_range=(1,2))),
#     ('clf',RandomForestClassifier(random_state=0, n_jobs=-1, n_estimators=500)),
    ('clf',MultiOutputClassifier(RandomForestClassifier(random_state=1234, n_jobs=-1, n_estimators=100)))
])

model_m.fit(X_train.message, y_train)

# predict on test data
y_pred_model_m = model_m.predict(X_test.message)

# show result
show_result(y_pred_model_m)

//anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


related 
               precision    recall  f1-score   support

           0       0.73      0.35      0.48      1563
           1       0.82      0.96      0.88      4920
           2       0.50      0.06      0.10        53

    accuracy                           0.81      6536
   macro avg       0.68      0.46      0.49      6536
weighted avg       0.79      0.81      0.78      6536

request 
               precision    recall  f1-score   support

           0       0.90      0.98      0.94      5428
           1       0.84      0.49      0.62      1108

    accuracy                           0.90      6536
   macro avg       0.87      0.74      0.78      6536
weighted avg       0.89      0.90      0.89      6536

offer 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6516
           1       0.00      0.00      0.00        20

    accuracy                           1.00      6536
   macro avg       0.50      0.50      0.50   

//anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


shops 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6511
           1       0.00      0.00      0.00        25

    accuracy                           1.00      6536
   macro avg       0.50      0.50      0.50      6536
weighted avg       0.99      1.00      0.99      6536

aid_centers 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6471
           1       0.00      0.00      0.00        65

    accuracy                           0.99      6536
   macro avg       0.50      0.50      0.50      6536
weighted avg       0.98      0.99      0.99      6536

other_infrastructure 
               precision    recall  f1-score   support

           0       0.95      1.00      0.98      6239
           1       0.25      0.00      0.01       297

    accuracy                           0.95      6536
   macro avg       0.60      0.50      0.49      6536
weighted avg       0.92      

In [36]:
with open('../models/model.pkl', 'wb') as file:
    pickle.dump(model_m, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.